In [ ]:
# from itertools import chain
import sys
sys.path.append('..')
# import timm
# import pandas as pd
import torchvision.transforms as T
# from torch.optim import SGD

# from wildlife_tools.data import WildlifeDataset, SplitMetadata
# from wildlife_tools.train import ArcFaceLoss
# from wildlife_tools.train import BasicTrainer


# import timm
# import numpy as np
# from wildlife_datasets.datasets import WhaleSharkID
from wildlife_tools.data import WildlifeDataset
# import torchvision.transforms as T
# from wildlife_datasets import datasets, splits
# from wildlife_tools.features import DeepFeatures
# from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

# import timm
import itertools
from torch.optim import SGD
# from wildlife_tools.train import ArcFaceLoss, BasicTrainer , TripletLoss
from utils.trainer_pl import basic_trainer_pl
from models.template_model import TemplateModel
from utils.triplet_loss_utils import TripletLoss_wildlife


import sys
sys.path.append('..')
import argparse
import shutil
import os
import yaml
import timm
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_lightning import Trainer
import numpy as np
from PIL import Image
import wandb
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from wildlife_datasets import analysis, datasets, loader

In [11]:
root = '/Users/amee/Documents/code/master-thesis/datasets/ATRW/'

# Load dataset metadata
metadata = datasets.ATRW(root)
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)



# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss_wildlife()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = basic_trainer_pl(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=0,
    device='cpu',
    epoch_callback=print_epoch_loss
)



GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/amee/miniconda3/envs/pytorch_env/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name        | Type                 | Params | Mode 
-------------------------------------------------------------
0 | model       | SwinTransformer      | 27.5 M | train
1 | loss_module | TripletLoss_wildlife | 0      | train
-------------------------------------------------------------
27.5 M    Trainable params
0         Non-trainable params
27.5 M    Total params
110.077   Total estimated model params size (MB)
/Users/amee/miniconda3/envs/pytorch_env/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
/Users/amee/miniconda3/env

In [12]:
from wildlife_tools.features import DeepFeatures


dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone , device = 'cpu')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 42/42 [05:46<00:00,  8.26s/it]


In [13]:
from utils.triplet_loss_utils import KnnClassifier
from wildlife_tools.similarity import CosineSimilarity

similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[ 0.02134773, -0.10590538,  0.02437633, ...,  0.03205836,
         0.04964858,  0.0191994 ],
       [ 0.08248976, -0.02898927, -0.04742883, ..., -0.00551586,
        -0.07753257,  0.0472618 ],
       [-0.07289029, -0.02074489,  0.14086668, ...,  0.05982637,
        -0.03907153, -0.08189144],
       ...,
       [ 0.03932605,  0.10952195,  0.02927184, ...,  0.15346071,
         0.08338239,  0.10213841],
       [ 0.00754523,  0.03353639,  0.08925048, ...,  0.00589957,
         0.05694124,  0.06781136],
       [ 0.11477285,  0.0210886 ,  0.03463744, ...,  0.01759844,
        -0.01871965,  0.02832869]], dtype=float32)}


In [14]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

100
5315
Predictions for 100 test Images:-
 ['16' '34' '177' '18' '112' '18' '19' '144' '177' '153' '205' '147' '208'
 '166' '27' '243' '109' '163' '169' '178' '270' '96' '12' '234' '250'
 '211' '156' '190' '4' '193' '240' '156' '57' '10' '112' '169' '103' '97'
 '261' '120' '169' '246' '172' '70' '0' '199' '267' '112' '156' '118' '15'
 '121' '162' '159' '159' '90' '141' '213' '48' '246' '166' '162' '252'
 '93' '73' '84' '18' '39' '244' '192' '268' '100' '166' '255' '244' '249'
 '31' '177' '48' '136' '162' '64' '220' '141' '207' '7' '162' '162' '84'
 '196' '21' '157' '147' '162' '177' '123' '156' '88' '238' '157']


/Users/amee/Documents/code/master-thesis/EagleID/notebooks/../utils/triplet_loss_utils.py:138: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [15]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on ATRW data: {:.2f}%".format(accuracy_P * 100))

Accuracy on ATRW data: 95.00%


In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.9883333333333333
Recall: 0.95
F1 Score: 0.9493333333333334
